In [1]:
import os
import pysal as ps 
import pandas as pd 
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pysal.lib
import pysal.explore
import pysal.viz
import pysal.model

C:\Users\jwolf\AppData\Local\conda\conda\envs\pandas\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
os.chdir('C:/Users/jan_lenovo/Documents/MEGA/03_Doutoramento/06_Base de dados/School closure')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/jan_lenovo/Documents/MEGA/03_Doutoramento/06_Base de dados/School closure'

In [ ]:
path = "C:/Users/jan_lenovo/Documents/MEGA/03_Doutoramento/05_SIG/Administrativos/Municípios/Cont_AAD_CAOP2018/\
Cont_AAD_CAOP2018.shp"
gdf_freguesias_CAOP2018 = gpd.read_file (path)
gdf_freguesias_CAOP2018['code_mun'] = gdf_freguesias_CAOP2018['Dicofre'].str.slice(0,4)

In [ ]:
df_correspond_nuts3 = pd.read_csv("CODIGOS_MUNICIPIOS_NUTS3.csv",engine='python', header = 0,\
                                  sep = ";",dtype= {'Cod_mun': str}, decimal=",")
df_correspond_nuts3.dropna(inplace = True)

In [ ]:
# pd.merge(gdf_freguesias_CAOP2018, df_correspond_nuts3, how = left,  left_on = "code_mun")
gdf_freguesias_CAOP2018= pd.merge(gdf_freguesias_CAOP2018, df_correspond_nuts3, how='left', left_on = "code_mun",right_on= "Cod_mun")

In [ ]:
#gdf_freguesias_CAOP2018.to_file("C:/Users/jwolf/Documents/MEGA/03_Doutoramento/05_SIG/Administrativos/", driver = 'GeoJSON')

In [ ]:
gdf_freguesias_CAOP2018.head(3)

In [ ]:
# gdf_freguesias_CAOP2018.plot(figsize = [10,10])
gdf_freguesias_CAOP2018.drop(['Cod_mun'], axis=1, inplace = True)

In [ ]:
#3763 código epsg etrs89 e Portugal TM06
#WGS84 = epsg 4326

gdf_freguesias_CAOP2018 = gdf_freguesias_CAOP2018.to_crs(epsg=3763)

In [ ]:
gdf_municipios_CAOP2018 = gdf_freguesias_CAOP2018.dissolve(by='code_mun')
gdf_municipios_CAOP2018.drop(['Dicofre','Freguesia', 'Des_Simpli', 'TAA'], axis = 1, inplace = True)

In [ ]:
df_schools = pd.read_csv("C:/Users/jan_lenovo/Documents/MEGA/03_Doutoramento/05_SIG/Muhamad/Schools.csv",\
                         engine='python', header = 0, sep = ";",dtype= {'Code': str}, decimal=",")
gdf_municipios_CAOP2018 = pd.merge(gdf_municipios_CAOP2018, df_schools, how='left', left_on=['code_mun'],\
                                   right_on = ['Code'])

In [ ]:
gdf_municipios_CAOP2018.head(2)

In [ ]:
df_TFR = pd.read_csv("C:/Users/jwolf/Documents/MEGA/03_Doutoramento/05_SIG/Muhamad/ISF_2017_pivot.csv",\
                         engine='python', header = 0, sep = ";",dtype= {'Code': str}, decimal=",")
gdf_municipios_CAOP2018 = pd.merge(gdf_municipios_CAOP2018, df_TFR, how='left', left_on=['Code'],\
                                   right_on = ['code'])

In [ ]:
gdf_municipios_CAOP2018.head(2)

In [ ]:
gdf_freguesias_CAOP2018.plot(figsize = (10,10))

In [ ]:
# w3= pysal.lib.weights.KNN.from_shapefile(path, k= 3)
matriz_dist_3_vizinhos= pysal.lib.weights.KNN.from_dataframe(gdf_municipios_CAOP2018, k= 3)

In [ ]:
w_distancia = pysal.lib.weights.DistanceBand.from_dataframe(gdf_municipios_CAOP2018, p = 2, alpha=-2, threshold= 30000)

In [ ]:
y = gdf_municipios_CAOP2018['TX_99_17']
y2= gdf_municipios_CAOP2018['2017']
moran_3 = pysal.explore.esda.Moran_Local(y, matriz_dist_3_vizinhos, transformation='r', permutations=999)
moran_dist = pysal.explore.esda.Moran_Local(y, w_distancia, transformation='r', permutations=999)
moran_3_TFR = pysal.explore.esda.Moran_Local(y2, matriz_dist_3_vizinhos, transformation='r', permutations=999)
moran_dist_TFR = pysal.explore.esda.Moran_Local(y2, w_distancia, transformation='r', permutations=999)

In [ ]:
moran_I = pysal.explore.esda.Moran(y2, w_distancia, transformation='r', permutations=999)


print("p value =", moran_I.p_norm, "Moran I = ", moran_I.I,"expected value under normality assumption", moran_I.EI) 

In [ ]:
from pysal.viz.splot.esda import lisa_cluster
gdf_municipios_CAOP2018_2 = gdf_municipios_CAOP2018.copy()

# fig, ax = plt.subplots()
# ax.plot(x, y)
# ax.set_title('Simple plot')

fig, axs = plt.subplots(1,2,figsize = (10,10))
cartogramas = [(moran_3_TFR, '3_neighbors'), (moran_dist_TFR, '20_km')]
i=0 
for x in cartogramas:
    legenda = {'loc': 'upper left', 'bbox_to_anchor': (0.8, 0.28), 'title': x[1]}
    lisa_cluster(x[0], gdf_municipios_CAOP2018_2, p=0.1, ax = axs [i], legend_kwds= legenda)
    i += 1


In [ ]:
gdf_municipios_CAOP2018.plot()

In [ ]:
w3= pysal.lib.weights.KNN.from_shapefile(path, k= 3)

In [ ]:
gdf_municipios_CAOP2018.columns

In [ ]:
gdf_municipios_CAOP2018

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 6))
# ax.axis('off')
gdf_freguesias_CAOP2018.plot(ax = ax,column='AREA_T_Ha')
# gdf_freguesias_CAOP2018.plot(ax = ax , linewidth=0, column = 0)